In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from lib.utils import non_max_suppression, intersection_over_union, load_checkpoint, cells_to_bboxes
from lib.utils import intersection_over_union as iou
from lib.YOLOV3 import YOLOv3 as YOLO
from albumentations.pytorch import ToTensorV2
import albumentations as A
from lib import config as C
import cv2
from torch.utils.data import DataLoader
from PIL import Image, ImageFile, ImageDraw, ImageFont
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from lib.utils import (
    cells_to_bboxes,
    iou_width_height as iou,
    non_max_suppression as nms,
    plot_image
)

C:\Users\dokee\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
file = open("lib/datasets/test.txt")
data = file.read()
test= data.replace("\n", " ").split()

In [5]:
annotations = pd.read_csv("lib/datasets/LISA/LISA/allAnnotations.csv", delimiter=";")
annotations

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,"Occluded,On another road",Origin file,Origin frame number,Origin track,Origin track frame number
0,aiua120214-0/frameAnnotations-DataLog02142012_...,stop,862,104,916,158,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2667,stop_1330545910.avi,2
1,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimitUrdbl,425,197,438,213,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2667,stop_1330545910.avi,2
2,aiua120214-0/frameAnnotations-DataLog02142012_...,stop,922,88,982,148,"1,0",aiua120214-0/DataLog02142012_external_camera.avi,2672,stop_1330545910.avi,7
3,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,447,193,461,210,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2672,stop_1330545910.avi,7
4,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,469,189,483,207,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2677,stop_1330545910.avi,12
...,...,...,...,...,...,...,...,...,...,...,...
7850,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,41,209,65,239,"0,0",vid9/MVI_0121.MOV,8813,speedLimit_1324866802.avi,22
7851,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,526,213,543,233,"0,0",vid9/MVI_0121.MOV,8875,speedLimit_1324866807.avi,2
7852,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,546,208,564,230,"0,0",vid9/MVI_0121.MOV,8880,speedLimit_1324866807.avi,7
7853,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,573,204,592,228,"0,0",vid9/MVI_0121.MOV,8885,speedLimit_1324866807.avi,12


In [6]:
df = annotations[annotations["Filename"].isin(test)]
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,"Occluded,On another road",Origin file,Origin frame number,Origin track,Origin track frame number
12,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,511,191,530,213,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2766,speedLimit_1330545914.avi,51
13,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,512,192,531,216,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2773,speedLimit_1330545914.avi,58
14,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,513,193,532,217,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2780,speedLimit_1330545914.avi,65
20,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,551,191,572,216,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2822,speedLimit_1330545914.avi,107
27,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,719,162,746,194,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2871,speedLimit_1330545914.avi,156
...,...,...,...,...,...,...,...,...,...,...,...
7841,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,167,208,187,232,"0,0",vid9/MVI_0121.MOV,8376,speedLimit_1324866786.avi,22
7843,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,117,192,141,223,"0,0",vid9/MVI_0121.MOV,8386,speedLimit_1324866786.avi,32
7845,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,32,164,68,207,"0,0",vid9/MVI_0121.MOV,8396,speedLimit_1324866786.avi,42
7846,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,149,232,164,251,"0,0",vid9/MVI_0121.MOV,8793,speedLimit_1324866802.avi,2


In [8]:
def get_eval_boxes(x, model, anchors, iou_threshold, threshold, device="cuda"):
    model.eval()
    x = x.to(device)
    tmp = torch.reshape(x,(1,x.size()[0],x.size()[1],x.size()[2]))
    all_pred_boxes = []
    train_idx = 0
    with torch.no_grad():
        preditcions = model(tmp)
    batch_size = tmp.shape[0]
    bboxes = bboxes = [[] for _ in range(batch_size)]
    for i in range(3):
        S = preditcions[i].shape[2]
        anchor = torch.tensor([*anchors[i]]).to(device) * S
        boxes_scale_i = cells_to_bboxes(
            preditcions[i], anchor, S = S, is_preds=True

        )
        for idx, (box) in enumerate(boxes_scale_i):
            bboxes[idx] += box

    for idx in range(batch_size):
        nms_boxes = non_max_suppression(
            bboxes[idx],
            iou_threshold = iou_threshold,
            threshold = threshold,
            box_format = "midpoint"
        )

        for nms_box in nms_boxes:
            all_pred_boxes.append([train_idx] + nms_box)
            train_idx += 1
    model.train()
    return all_pred_boxes

In [7]:
test_transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=C.IMAGE_SIZE),
        A.PadIfNeeded(
            min_height=int(C.IMAGE_SIZE),
            min_width=int(C.IMAGE_SIZE),
            border_mode=cv2.BORDER_CONSTANT,
        ),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255, ),
        ToTensorV2(),
    ],
)

In [7]:
path = "lib/datasets/LISA/LISA/"
S=[C.IMAGE_SIZE // 32, C.IMAGE_SIZE // 16, C.IMAGE_SIZE // 8]
anchors = C.ANCHORS
check = "lib/models/checkpoint_test.pth.tar"
model = YOLO(num_classes=C.NUM_CLASSES).to(C.DEVICE)
optimizer = optim.Adam(
        model.parameters(), lr=C.LEARNING_RATE, weight_decay=C.WEIGHT_DECAY
    )
load_checkpoint(
        check, model, optimizer, C.LEARNING_RATE
    )
ids = pd.read_csv("lib/datasets/ids.csv")

=> Loading checkpoint


In [25]:
pred_names = []
pred_boxes = []
pred_scores = []
for i in test:
    image_path = path + i
    image = np.array(Image.open(image_path).convert("RGB"))
    augmentations = test_transforms(image=image)
    image1 = augmentations["image"]
    bboxes = get_eval_boxes(image1, model, C.ANCHORS, iou_threshold = C.NMS_IOU_THRESH, threshold=C.CONF_THRESHOLD)
    image = Image.open(image_path)
    width, height = image.size
    ratio_height = height * C.IMAGE_SIZE / width
    convert_height = (416 - ratio_height) / 2
    tmp_names = []
    tmp_boxes = []
    tmp_scores = []
    for count, box in enumerate(bboxes):
        tmp_scores.append(box[2])
        name = ids[ids["ID"] == int(box[1])]["Name"].tolist()[0]
        tmp_names.append(name)
        x = box[3] * C.IMAGE_SIZE
        y = box[4] * C.IMAGE_SIZE
        y -= convert_height
        x, y = x / 416 * width,  y / ratio_height * height
        w = box[5] * width
        h = box[6] * width
        left, right, top, bottom = [(x - w / 2), (x + w / 2),
                                    (y - h / 2), (y + h / 2)]
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)
        tmp_boxes.append([left, top, right, bottom])
    print(len(tmp_names), len(tmp_boxes))
    pred_names.append(tmp_names)
    pred_boxes.append(tmp_boxes)
    pred_scores.append(tmp_scores)

1 1
2 2
1 1
1 1
1 1
1 1
2 2
1 1
1 1
2 2
2 2
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
3 3
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
2 2
1 1
1 1
1 1
1 1
2 2
2 2
3 3
1 1
1 1
1 1
1 1
1 1
2 2
2 2
3 3
1 1
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
1 1
2 2
1 1
1 1
2 2
1 1
3 3
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
2 2
2 2
3 3
1 1
2 2
1 1
2 2
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
3 3
1 1
1 1
4 4
2 2
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
3 3
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
3 3
1 1
1 1
1 1
1 1
1 1
2 2
2 2
3 3
1 1
2 2
1 1
1 1
1 1
1 1
2 2
1 1
3 3
1 1
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1
1 1
1 1
1 1
2 2
1 1
1 1


In [19]:
def IOU(true, pred):
    if true[0] > pred[0]:
        left = true[0]
    else:
        left = pred[0]

    if true[2] < pred[2]:
        right = true[2]
    else:
        right = pred[2]

    if true[1] > pred[1]:
        top = true[1]
    else:
        top = pred[1]

    if true[3] < pred[3]:
        bottom = true[3]
    else:
        bottom = pred[3]

    x = right - left
    y = bottom - top
    area_overlap = x * y
    if area_overlap < 0:
        area_overlap = 0
    area_union = (true[2] - true[0]) * (true[3] - true[1]) + (pred[2] - pred[0]) * (pred[3] - pred[1]) - area_overlap
    return area_overlap / area_union

In [17]:
thresholds = np.arange(start=0.6, stop = 1.05, step=0.05)
thresholds

array([0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [ ]:
precision = np.zeros(7)
recall = np.zeros(7)

In [12]:
groups = df.groupby("Filename")
true_names = []
true_boxes = []
for i in test:
    group = groups.get_group(i)
    tmp_names = []
    tmp_boxes = []
    for index, row in group.iterrows():
        tmp_names.append(row["Annotation tag"])
        tmp_boxes.append([row["Upper left corner X"], row["Upper left corner Y"], row["Lower right corner X"], row["Lower right corner Y"]])
    true_names.append(tmp_names)
    true_boxes.append(tmp_boxes)

In [36]:
tp_class = np.zeros(47)
fp_class = np.zeros(47)
fn_class = np.zeros(47)
total_true = np.zeros(47)
threshold = 0.7
for i in range(len(true_names)):
    tnames = true_names[i]
    tboxes = true_boxes[i]
    pnames = pred_names[i]
    pboxes = pred_boxes[i]
    pscores = pred_scores[i]
    for j in tnames:
        index = ids[ids["Name"]==j]["ID"].tolist()[0]
        total_true[index] += 1
    for j in range(len(pnames)):
        tmp = True
        index = ids[ids["Name"]==pnames[j]]["ID"].tolist()[0]
        for k in range(len(tnames)):
            iou = IOU(tboxes[k], pboxes[j])
            if iou > threshold and tnames[k] == pnames[j]:
                tp_class[index] += 1
                tmp = False
            elif iou > 0:
                fp_class[index] += 1
                tmp = False
        if tmp:
            fn_class[index] += 1

In [37]:
tp_class, fp_class, fn_class, total_true

(array([ 52.,   5.,   8.,   6.,   3.,   6.,   1.,  66.,  41.,  64.,   8.,
          5., 207.,   3.,   1.,   0.,   6.,   3.,   0.,  20.,   8.,  31.,
         12., 163.,   6.,   3.,  54.,  18.,  94.,  16.,  40.,  12.,   0.,
         23.,  31., 335.,  28.,   1.,   1.,   6.,   7.,   4.,  21.,  31.,
         10.,   1.,   6.]),
 array([ 8.,  1.,  3.,  1.,  1.,  0.,  0., 23., 10.,  5.,  2.,  1., 24.,
         1.,  0.,  0.,  1.,  0.,  0.,  3.,  2.,  7.,  6., 26.,  4.,  0.,
         3.,  1., 16.,  0.,  3.,  9.,  0.,  0.,  6., 48.,  3.,  0.,  0.,
         1.,  2.,  3.,  0., 17.,  3.,  3.,  0.]),
 array([1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 2., 0., 5., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 7., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        3., 6., 1., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.]),
 array([ 57.,   5.,   9.,   6.,   3.,   6.,   1.,  69.,  41.,  66.,   9.,
          7., 212.,   3.,   1.,   0.,   7.,   3.,   0.,  21.,   9.,  33.,
         14., 171.,   6.,   3.,  58.,  18., 1

In [40]:
ap = []
epsilon = 1e-6
for i in range(len(tp_class)):
    if total_true[i] != 0:
        recall = tp_class[i] / (total_true[i] + epsilon)
        precision = tp_class[i] / (tp_class[i] + fp_class[i] + epsilon)
        ap.append(np.trapz([1,precision], [0,recall]))
sum(ap) / len(ap)

0.8639858142319327

In [10]:
def get_eval_boxes(x, model, anchors, iou_threshold, threshold, device="cuda"):
    model.eval()
    x = x.to(device)
    tmp = torch.reshape(x,(1,x.size()[0],x.size()[1],x.size()[2]))
    all_pred_boxes = []
    train_idx = 0
    with torch.no_grad():
        preditcions = model(tmp)
    batch_size = tmp.shape[0]
    bboxes = bboxes = [[] for _ in range(batch_size)]
    for i in range(3):
        S = preditcions[i].shape[2]
        anchor = torch.tensor([*anchors[i]]).to(device) * S
        boxes_scale_i = cells_to_bboxes(
            preditcions[i], anchor, S = S, is_preds=True

        )
        for idx, (box) in enumerate(boxes_scale_i):
            bboxes[idx] += box

    for idx in range(batch_size):
        nms_boxes = non_max_suppression(
            bboxes[idx],
            iou_threshold = iou_threshold,
            threshold = threshold,
            box_format = "midpoint"
        )

        for nms_box in nms_boxes:
            all_pred_boxes.append([train_idx] + nms_box)
            train_idx += 1
    model.train()
    return all_pred_boxes

In [13]:
# test_transforms = A.Compose(
#         [
#             A.LongestMaxSize(max_size=C.IMAGE_SIZE),
#             A.PadIfNeeded(
#                 min_height=int(C.IMAGE_SIZE),
#                 min_width=int(C.IMAGE_SIZE),
#                 border_mode=cv2.BORDER_CONSTANT,
#             ),
#             A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255, ),
#             ToTensorV2(),
#         ],
#     )
# S=[C.IMAGE_SIZE // 32, C.IMAGE_SIZE // 16, C.IMAGE_SIZE // 8]
# anchors = C.ANCHORS
# model = YOLO(num_classes=C.NUM_CLASSES).to(C.DEVICE)
# check = "lib/models/checkpoint_test.pth.tar"
# optimizer = optim.Adam(
#     model.parameters(), lr=C.LEARNING_RATE, weight_decay=C.WEIGHT_DECAY
# )
# load_checkpoint(
#     check, model, optimizer, C.LEARNING_RATE
# )
# image_path = path + df["Filename"].tolist()[0]
# image = np.array(Image.open(image_path).convert("RGB"))
# augmentations = test_transforms(image=image)
# image1 = augmentations["image"]
import time
start = time.time()
bboxes = get_eval_boxes(image1, model, C.ANCHORS, iou_threshold = C.NMS_IOU_THRESH, threshold=C.CONF_THRESHOLD)
print(time.time() - start)

0.20942211151123047
